## CREDICT DEFAULTERS PREDICTION USING KERAS IN PYTHON

The dataset belongs to TW banking system. Our aim here is to predict the credit defaulters with certain known parameters.

# Drive Mounting.

In [6]:
# Importing the required libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras
import itertools
%matplotlib inline

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
os.chdir('/content/drive/MyDrive/DeepL/8. Credit Card Default Prediction')

In [5]:
!dir

creditcarddefault.csv
Keras_Problem\ Statement\ (1).pdf
Keras_Problem\ Statement.pdf
TW\ Credit\ Defaulters\ Prediction_with\ Keras\ (1).ipynb
TW\ Credit\ Defaulters\ Prediction_with\ Keras\ (2).ipynb
TW\ Credit\ Defaulters\ Prediction_with\ Keras.ipynb


In [8]:
# importing the dataset
TW_data = pd.read_csv("creditcarddefault.csv")

In [10]:
TW_data.tail(5)

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
29995,29996,220000,1,3,1,39,0,0,0,0,...,88004,31237,15980,8500,20000,5003,3047,5000,1000,0
29996,29997,150000,1,3,2,43,-1,-1,-1,-1,...,8979,5190,0,1837,3526,8998,129,0,0,0
29997,29998,30000,1,2,2,37,4,3,2,-1,...,20878,20582,19357,0,0,22000,4200,2000,3100,1
29998,29999,80000,1,3,1,41,1,-1,0,0,...,52774,11855,48944,85900,3409,1178,1926,52964,1804,1
29999,30000,50000,1,2,1,46,0,0,0,0,...,36535,32428,15313,2078,1800,1430,1000,1000,1000,1


In [9]:
TW_data.describe()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
count,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,30000.000000,...,30000.000000,30000.000000,30000.000000,30000.000000,3.000000e+04,30000.00000,30000.000000,30000.000000,30000.000000,30000.000000
mean,15000.500000,167484.322667,1.603733,1.853133,1.551867,35.485500,-0.016700,-0.133767,-0.166200,-0.220667,...,43262.948967,40311.400967,38871.760400,5663.580500,5.921163e+03,5225.68150,4826.076867,4799.387633,5215.502567,0.221200
std,8660.398374,129747.661567,0.489129,0.790349,0.521970,9.217904,1.123802,1.197186,1.196868,1.169139,...,64332.856134,60797.155770,59554.107537,16563.280354,2.304087e+04,17606.96147,15666.159744,15278.305679,17777.465775,0.415062
min,1.000000,10000.000000,1.000000,0.000000,0.000000,21.000000,-2.000000,-2.000000,-2.000000,-2.000000,...,-170000.000000,-81334.000000,-339603.000000,0.000000,0.000000e+00,0.00000,0.000000,0.000000,0.000000,0.000000
25%,7500.750000,50000.000000,1.000000,1.000000,1.000000,28.000000,-1.000000,-1.000000,-1.000000,-1.000000,...,2326.750000,1763.000000,1256.000000,1000.000000,8.330000e+02,390.00000,296.000000,252.500000,117.750000,0.000000
50%,15000.500000,140000.000000,2.000000,2.000000,2.000000,34.000000,0.000000,0.000000,0.000000,0.000000,...,19052.000000,18104.500000,17071.000000,2100.000000,2.009000e+03,1800.00000,1500.000000,1500.000000,1500.000000,0.000000
75%,22500.250000,240000.000000,2.000000,2.000000,2.000000,41.000000,0.000000,0.000000,0.000000,0.000000,...,54506.000000,50190.500000,49198.250000,5006.000000,5.000000e+03,4505.00000,4013.250000,4031.500000,4000.000000,0.000000
max,30000.000000,1000000.000000,2.000000,6.000000,3.000000,79.000000,8.000000,8.000000,8.000000,8.000000,...,891586.000000,927171.000000,961664.000000,873552.000000,1.684259e+06,896040.00000,621000.000000,426529.000000,528666.000000,1.000000


## UNDERSTANDING THE DATA
The first column mentions the customer ID. The IDs are unique and we will not need it to predict the outcome as it does not provide any information regarding the credit defaulting.Hence let us consider only the rest of the data. 

There are 23 other independent variables which are a combination of categoriical and numeric variables. The dependent varaible is binary shows whether the customer is a credit defaulter or not.

As the dependent variable is categorical this becomes a classification problem. In this exercise we use Keras to build an ANN in order to predict the credit defaulters.

As a first step we deeply take a look at the data and perform our EDA before proceeding with the modelling

In [11]:
# removing the customer id and name and saving the dependent varaibles 1n x
x = TW_data.iloc[:,1:24].values

# saving the independent variable in y
y = TW_data.iloc[:,24].values

print(x)
print(y)

[[ 20000      2      2 ...      0      0      0]
 [120000      2      2 ...   1000      0   2000]
 [ 90000      2      2 ...   1000   1000   5000]
 ...
 [ 30000      1      2 ...   4200   2000   3100]
 [ 80000      1      3 ...   1926  52964   1804]
 [ 50000      1      2 ...   1000   1000   1000]]
[1 1 0 ... 1 1 1]


In [12]:
x.shape

(30000, 23)

In [13]:
y.shape

(30000,)

In [15]:
# Splitting the data set into train and test using train_test_split library for cross validation

from sklearn.model_selection import train_test_split

# splitting the data in 70:30 ratio
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 111)
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(21000, 23)
(21000,)
(9000, 23)
(9000,)


In [16]:
# scaling the data using the preprocessing library 
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [17]:
x_train[:1]

array([[ 0.48238852,  0.81447304,  0.18907673,  0.86211952, -0.48921975,
        -0.87463676, -0.72347864, -0.6983838 , -0.66637051,  0.23416564,
         0.25233359, -0.63042993, -0.63367788, -0.6645842 , -0.51248219,
        -0.57038965,  0.39521137, -0.0821721 , -0.22758026,  0.33473587,
        -0.28260403,  3.38918769, -0.15149706]])

## MODEL BUILDING USING KERAS

We have so far understood the data and made some preprocessing on the data in order to make the data suitable for our model. Now that our data is ready to be trained let us build our first Keras model.

We are here using a sequential model from the Keras as the data is simple.

In [18]:
# Importing the required modules from keras

from keras.models import Sequential
from keras.layers import Dense

In [19]:
# STEP 1:  initializing our model
class_model_keras = Sequential()

In [30]:
# STEP 2: Adding the layers to our NN. 
# In this step we add 1 i/p layer(which are the ips directly), multiple hidden layers & 1 o/p layer
# As there are 23 input variables there will be 23 nodes in the input layer
# Nodes in the hidden layer is free to our choice, however to have an optinum error
# we can calculate the nodes in our first hidden layer to be (I/p nodes + 1 )/2 = (23+1)/2 = 12

# While initializing random weights to the NN, 
# we pass the value to the hyperparameter init as "UNIFORM"
# UNIFORM will ensure that the weights are given uniformly random and close to 0

# Also we would be specifying what activation function to be used. LEt us use RELU in our model here


# 1st HIDDEN LAYER
class_model_keras.add(Dense(units= 128, activation = 'relu' ))

# 2nd Hidden Layer 
# as the input dim to this layer is the output from the previous layer 
# we need not explicitly specify it here
class_model_keras.add(Dense(units = 12, activation = 'relu'))

# OUTPUT LAYER
# sigmoid activation is used to get the outout between 0 and 1.
class_model_keras.add(Dense(units = 12, activation = 'sigmoid'))

In [34]:
# STEP 3: COMPILING THE NEURAL NETWORK
# In this step we have the liberty to choose the optimization method we would like to use 
# the loss fucntion and the metrics that we require ad outpur
# binary_crossentropy loss function used when a binary output is expected

#class_model_keras.compile(optimizer='sgd', loss = 'binary_crossentropy', metrics = ['accuracy'])
class_model_keras.compile(optimizer='adam',loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])

In [35]:
# STEP 4: FITTING THE MODEL

class_model_keras.fit(x_train, y_train, batch_size = 10, epochs = 50)

Epoch 1/50


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


2100/2100 [==============================] - 5s 2ms/step - loss: 0.5102 - accuracy: 0.8039
Epoch 2/50
2100/2100 [==============================] - 4s 2ms/step - loss: 0.4414 - accuracy: 0.8184
Epoch 3/50
2100/2100 [==============================] - 4s 2ms/step - loss: 0.4359 - accuracy: 0.8215
Epoch 4/50
2100/2100 [==============================] - 4s 2ms/step - loss: 0.4326 - accuracy: 0.8199
Epoch 5/50
2100/2100 [==============================] - 4s 2ms/step - loss: 0.4314 - accuracy: 0.8204
Epoch 6/50
2100/2100 [==============================] - 5s 2ms/step - loss: 0.4288 - accuracy: 0.8193
Epoch 7/50
2100/2100 [==============================] - 4s 2ms/step - loss: 0.4265 - accuracy: 0.8200
Epoch 8/50
2100/2100 [==============================] - 4s 2ms/step - loss: 0.4253 - accuracy: 0.8218
Epoch 9/50
2100/2100 [==============================] - 4s 2ms/step - loss: 0.4227 - accuracy: 0.8222
Epoch 10/50
2100/2100 [==============================] - 4s 2ms/step - loss: 0.4216 - accurac

In [37]:
# STEP 5: PREDICTING THE RESULTS FOR TEST DATA

y_pred =  class_model_keras.predict(x_test)

In [39]:
y_pred

array([[7.8748047e-01, 4.8969537e-01, 1.6435297e-13, ..., 2.6036939e-13,
        1.4889203e-13, 1.5360315e-13],
       [8.5789406e-01, 4.8574582e-01, 4.1841653e-18, ..., 1.4911484e-17,
        3.3609300e-18, 4.4524920e-18],
       [7.2762692e-01, 4.8847169e-01, 1.1506688e-10, ..., 1.1033586e-10,
        1.1163668e-10, 9.8224442e-11],
       ...,
       [9.1786623e-01, 4.8076564e-01, 6.6921516e-24, ..., 6.6326838e-23,
        4.6185554e-24, 8.3883966e-24],
       [7.4260855e-01, 6.2646663e-01, 1.4251665e-14, ..., 2.3885399e-14,
        1.5262239e-14, 1.9000652e-14],
       [9.5296502e-01, 4.5989168e-01, 1.7739822e-30, ..., 5.6123224e-29,
        1.0053395e-30, 2.5659908e-30]], dtype=float32)

In [38]:
# As the values are the probabality we may need to set up a threshold to find the actual values

pred = (y_pred> 0.5)

In [40]:
target_names = [ 'no', 'yes']
# code from http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True')
    plt.xlabel('Predicted')

In [44]:
# Evaluating the Model

scores = class_model_keras.evaluate(x_test,y_test)

print("\nACCURACY: %.2f%%" % (scores[1]*100))

  1/282 [..............................] - ETA: 1:50 - loss: 0.4180 - accuracy: 0.8125

/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


282/282 [==============================] - 1s 4ms/step - loss: 0.6712 - accuracy: 0.7973

ACCURACY: 79.73%


# CONCLUSION

Thus we have built our first model with Keras with 80 % accuracy